# Using data.gouv's metadata

In [48]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from descartes import PolygonPatch
import geopandas as gpd
from shapely.geometry import Point, Polygon
import shapefile
import folium
from scipy.spatial import ConvexHull


pd.set_option('expand_frame_repr', False)

In [49]:
metadata_df = pd.read_excel("../data/raw/fr-2023-d-lcsqa-ineris-20230717.xls", sheet_name="AirQualityStations")
df = pd.read_csv("../data/raw/FR_E2_2023-01-01.csv", delimiter=";")
df = df.rename(str.lower, axis='columns')

cross_df = df.merge(metadata_df[["NatlStationCode", "Latitude", "Longitude"]], left_on="code site", right_on="NatlStationCode")

cross_df = cross_df[(cross_df["Latitude"].between(-10, 60)) & 
                    (cross_df["Longitude"].between(-10, 60))]
display(cross_df.isna().sum())
display(cross_df)

date de début                0
date de fin                  0
organisme                    0
code zas                     0
zas                          0
code site                    0
nom site                     0
type d'implantation          0
polluant                     0
type d'influence             0
discriminant              6480
réglementaire                0
type d'évaluation            0
procédure de mesure          0
type de valeur               0
valeur                    1475
valeur brute              1475
unité de mesure              0
taux de saisie           46320
couverture temporelle    46320
couverture de données    46320
code qualité                 0
validité                     0
NatlStationCode              0
Latitude                     0
Longitude                    0
dtype: int64

,date de début,date de fin,organisme,code zas,zas,code site,nom site,type d'implantation,polluant,type d'influence,...,valeur brute,unité de mesure,taux de saisie,couverture temporelle,couverture de données,code qualité,validité,NatlStationCode,Latitude,Longitude
0,2023/01/01 00:00:00,2023/01/01 01:00:00,ATMO GRAND EST,FR44ZAG02,ZAG METZ,FR01011,Metz-Centre,Urbaine,NO,Fond,...,0.725,µg-m3,NaN,NaN,NaN,A,1,FR01011,49.119442,6.180833
1,2023/01/01 01:00:00,2023/01/01 02:00:00,ATMO GRAND EST,FR44ZAG02,ZAG METZ,FR01011,Metz-Centre,Urbaine,NO,Fond,...,0.700,µg-m3,NaN,NaN,NaN,A,1,FR01011,49.119442,6.180833
2,2023/01/01 02:00:00,2023/01/01 03:00:00,ATMO GRAND EST,FR44ZAG02,ZAG METZ,FR01011,Metz-Centre,Urbaine,NO,Fond,...,0.575,µg-m3,NaN,NaN,NaN,A,1,FR01011,49.119442,6.180833
3,2023/01/01 03:00:00,2023/01/01 04:00:00,ATMO GRAND EST,FR44ZAG02,ZAG METZ,FR01011,Metz-Centre,Urbaine,NO,Fond,...,0.800,µg-m3,NaN,NaN,NaN,A,1,FR01011,49.119442,6.180833
4,2023/01/01 04:00:00,2023/01/01 05:00:00,ATMO GRAND EST,FR44ZAG02,ZAG METZ,FR01011,Metz-Centre,Urbaine,NO,Fond,...,0.725,µg-m3,NaN,NaN,NaN,A,1,FR01011,49.119442,6.180833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50131,2023/01/01 19:00:00,2023/01/01 20:00:00,ATMO BOURGOGNE-FRANCHE-COMTE,FR27ZRE01,ZR BOURGOGNE-FRANCHE-COMTE,FR82070,Baume-les-Dames,Urbaine,PM2.5,Trafic,...,18.625,µg-m3,NaN,NaN,NaN,A,1,FR82070,47.351288,6.359722
50132,2023/01/01 20:00:00,2023/01/01 21:00:00,ATMO BOURGOGNE-FRANCHE-COMTE,FR27ZRE01,ZR BOURGOGNE-FRANCHE-COMTE,FR82070,Baume-les-Dames,Urbaine,PM2.5,Trafic,...,19.200,µg-m3,NaN,NaN,NaN,A,1,FR82070,47.351288,6.359722
50133,2023/01/01 21:00:00,2023/01/01 22:00:00,ATMO BOURGOGNE-FRANCHE-COMTE,FR27ZRE01,ZR BOURGOGNE-FRANCHE-COMTE,FR82070,Baume-les-Dames,Urbaine,PM2.5,Trafic,...,13.325,µg-m3,NaN,NaN,NaN,A,1,FR82070,47.351288,6.359722
50134,2023/01/01 22:00:00,2023/01/01 23:00:00,ATMO BOURGOGNE-FRANCHE-COMTE,FR27ZRE01,ZR BOURGOGNE-FRANCHE-COMTE,FR82070,Baume-les-Dames,Urbaine,PM2.5,Trafic,...,19.050,µg-m3,NaN,NaN,NaN,A,1,FR82070,47.351288,6.359722


In [50]:
map = folium.Map(location=[47, 2.2137], zoom_start=6)
markers = cross_df.drop_duplicates(subset="nom site").reset_index()
display(markers.shape)
for point in range(0, len(markers)):
    folium.CircleMarker((markers.at[point, "Latitude"], markers.at[point, "Longitude"]), 
                        tooltip=markers.at[point, "nom site"], 
                        radius=1).add_to(map)
map

(493, 27)

In [51]:
def create_convexhull_polygon(map_object, list_of_points, layer_name, line_color, fill_color, weight, text): 

    # Since it is pointless to draw a convex hull polygon around less than 3 points check len of input
    if len(list_of_points) < 3:
        return

    # Create the convex hull using scipy.spatial 
    form = [list_of_points[i] for i in ConvexHull(list_of_points).vertices]

    # Create feature group, add the polygon and add the feature group to the map 
    fg = folium.FeatureGroup(name=layer_name)
    fg.add_child(folium.vector_layers.Polygon(locations=form, color=line_color, fill_color=fill_color,
                                              weight=weight, popup=(folium.Popup(text))))
    map_object.add_child(fg)

    return(map_object)

create_convexhull_polygon(
    map,
    cross_df[["Latitude", "Longitude"]].apply(lambda x: x * 1).values.tolist(),
    "test", "lightblue", "lightskyblue", 5, "test text"
)